# Amazon Sentiment Review

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#NLTK libraries
import nltk
import re
import string
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from imblearn.over_sampling import SMOTE
# Machine Learning libraries
import sklearn 
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn import svm, datasets
from sklearn import preprocessing 


#Metrics libraries
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

#Visualization libraries
import matplotlib.pyplot as plt 
from matplotlib import rcParams
import seaborn as sns
from textblob import TextBlob
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import iplot
%matplotlib inline

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Other miscellaneous libraries
from scipy import interp
from itertools import cycle
import cufflinks as cf
from collections import defaultdict
from collections import Counter
from imblearn.over_sampling import SMOTE

# Import Data File

In [18]:
reviews = pd.read_csv('data/Musical_instruments_reviews.csv')
reviews

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


## Reviews File Attributes

In [19]:
reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


#### Description of columns in the file:

reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B <br />
asin - ID of the product, e.g. 0000013714 <br />
reviewerName - name of the reviewer <br />
helpful - helpfulness rating of the review, e.g. 2/3 <br />
reviewText - text of the review <br />
overall - rating of the product <br />
summary - summary of the review <br />
unixReviewTime - time of the review (unix time) <br />
reviewTime - time of the review (raw)

In [20]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB


## Preprocessing

#### Finding null values

In [21]:
reviews.isnull().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

#### Replace 'reviewText' null fields with the string 'missing'

In [22]:
reviews['reviewText'] = reviews['reviewText'].fillna('Missing')

#### The other column with null values is the reviewerName column.  Since we have a reviewerID column (with no null values), we can drop the reviewerName column as redundant

In [23]:
reviews.dropna(axis = 1, inplace = True)
reviews.isnull().sum()

reviewerID        0
asin              0
helpful           0
reviewText        0
overall           0
summary           0
unixReviewTime    0
reviewTime        0
dtype: int64

In [24]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   helpful         10261 non-null  object 
 3   reviewText      10261 non-null  object 
 4   overall         10261 non-null  float64
 5   summary         10261 non-null  object 
 6   unixReviewTime  10261 non-null  int64  
 7   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 641.4+ KB


In [25]:
reviews

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


# Nature of Sentiments

The sentiments are derived from the overall column in the following manner:
1.  A score of < 3 is Negative
2.  A score of > 3 is Positive
3.  A score equal to 3 is Neutral

In [26]:
reviews['overall'].value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

Since there is not a column with these sentiments, we'll create one from the 'overall' column that will serve as a label for each review

In [27]:
def label(rating):
    if rating['overall'] == 3:
        value = 'Neutral'
    elif rating['overall'] < 3:
        value = 'Negative'
    else: 
        value = 'Positive'
    return value

In [28]:
# Using the function to create a new column
reviews['Sentiment'] = reviews.apply(label, axis=1)
reviews.head()

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Sentiment
0,A2IBPI20UZIR0U,1384719342,"[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014",Positive
1,A14VAT5EAX3D9S,1384719342,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013",Positive
2,A195EZSQDW3E21,1384719342,"[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013",Positive
3,A2C00NNG1ZQQG2,1384719342,"[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",Positive
4,A94QU4C90B1AX,1384719342,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014",Positive


In [29]:
print('Breakdown of sentiments:')
reviews['Sentiment'].value_counts()

Breakdown of sentiments:


Positive    9022
Neutral      772
Negative     467
Name: Sentiment, dtype: int64

## Combine 'reviewText' and 'summary' columns

Combining these columns is a good idea since they will both have the same sentiment

In [30]:
reviews['reviews'] = reviews['reviewText'] + reviews['summary']
reviews.drop(['reviewText', 'summary'], axis = 1, inplace = True)
reviews.head()

,reviewerID,asin,helpful,overall,unixReviewTime,reviewTime,Sentiment,reviews
0,A2IBPI20UZIR0U,1384719342,"[0, 0]",5.0,1393545600,"02 28, 2014",Positive,"Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,1384719342,"[13, 14]",5.0,1363392000,"03 16, 2013",Positive,The product does exactly as it should and is q...
2,A195EZSQDW3E21,1384719342,"[1, 1]",5.0,1377648000,"08 28, 2013",Positive,The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,1384719342,"[0, 0]",5.0,1392336000,"02 14, 2014",Positive,Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,1384719342,"[0, 0]",5.0,1392940800,"02 21, 2014",Positive,This pop filter is great. It looks and perform...


## Creating year, month and day columns

In [31]:
#splitting the reviewTime column into yyyy mm dd columns
date_fix = reviews["reviewTime"].str.split(",", n = 1, expand = True) 
reviews['date'] = date_fix[0]
reviews['year'] = date_fix[1]
md_fix = reviews['date'].str.split(' ', n = 1, expand = True)
reviews['month'] = md_fix[0]
reviews['day'] = md_fix[1]
#delete reviewTime and temporary date colummn
reviews.drop(['reviewTime', 'date'], axis=1, inplace = True)
reviews.head()

,reviewerID,asin,helpful,overall,unixReviewTime,Sentiment,reviews,year,month,day
0,A2IBPI20UZIR0U,1384719342,"[0, 0]",5.0,1393545600,Positive,"Not much to write about here, but it does exac...",2014,02,28
1,A14VAT5EAX3D9S,1384719342,"[13, 14]",5.0,1363392000,Positive,The product does exactly as it should and is q...,2013,03,16
2,A195EZSQDW3E21,1384719342,"[1, 1]",5.0,1377648000,Positive,The primary job of this device is to block the...,2013,08,28
3,A2C00NNG1ZQQG2,1384719342,"[0, 0]",5.0,1392336000,Positive,Nice windscreen protects my MXL mic and preven...,2014,02,14
4,A94QU4C90B1AX,1384719342,"[0, 0]",5.0,1392940800,Positive,This pop filter is great. It looks and perform...,2014,02,21


In [32]:
# Count of reviews by year
reviews['year'].value_counts()

 2013    4055
 2014    2679
 2012    1936
 2011    1007
 2010     350
 2009     128
 2008      63
 2007      22
 2006      10
 2004       7
 2005       4
Name: year, dtype: int64

In [35]:
# Sentiments grouped by year
reviews.groupby(['year', 'Sentiment']).size()

year   Sentiment
 2004  Positive        7
 2005  Positive        4
 2006  Negative        1
       Neutral         1
       Positive        8
 2007  Negative        1
       Positive       21
 2008  Negative        2
       Neutral         7
       Positive       54
 2009  Negative        5
       Neutral         8
       Positive      115
 2010  Negative       16
       Neutral        20
       Positive      314
 2011  Negative       46
       Neutral        84
       Positive      877
 2012  Negative       85
       Neutral       130
       Positive     1721
 2013  Negative      170
       Neutral       319
       Positive     3566
 2014  Negative      141
       Neutral       203
       Positive     2335
dtype: int64

## Create 'helpful_review' column

The 'helpful' column contains 2 integers, representing a out b people who found the review helpful.  This statistic will best be represented in our model as a/b, so we will create a 'helpful_review' column that will show this ratio.

In [40]:
newreview = reviews["helpful"].str.split(",", n = 1, expand = True)
newreview1 = newreview[0].str.split('[', n = 1, expand = True)
newreview2 = newreview[1].str.split(']', n = 1, expand = True)
newreview1.reset_index(drop = True, inplace = True)
newreview2.reset_index(drop = True, inplace = True)

In [44]:
newreview1.drop([0], axis = 1, inplace = True)
newreview2.drop([1], axis = 1, inplace = True)

In [48]:
helpful_review = pd.concat([newreview1, newreview2], axis = 1)

In [49]:
helpful_review

,1,0
0,0,0
1,13,14
2,1,1
3,0,0
4,0,0
...,...,...
10256,0,0
10257,0,0
10258,0,0
10259,0,0


In [70]:
helpful_review[0] = helpful_review[0].astype(str).astype(int)
helpful_review[1] = helpful_review[1].astype(str).astype(int)
try:
    helpful_review['result'] = helpful_review[1] / helpful_review[0]
except ZeroDivisionError: 
    helpful_review['result'] = 0
helpful_review['result'].fillna(0, inplace = True)
helpful_review['result'] = helpful_review['result'].round(2)

reviews['helpful_review'] = helpful_review['result']
reviews.drop(['helpful'], axis = 1, inplace = True)

In [73]:
reviews.tail(200)

,reviewerID,asin,overall,unixReviewTime,Sentiment,reviews,year,month,day,helpful_review
10061,A3CUQ8L2S9ZW0H,B00BKXN9LS,5.0,1405036800,Positive,Works great. Easy to useFive Stars,2014,07,11,0.0
10062,A1UPZM995ZAH90,B00BKXN9LS,5.0,1390089600,Positive,My acoustic only has the end pin strap button ...,2014,01,19,0.0
10063,A1L7M2JXN4EZCR,B00BKXN9LS,5.0,1382054400,Positive,Leather type material and corrosive free butto...,2013,10,18,0.0
10064,A2Y2U697SS98E1,B00BKXN9LS,4.0,1402531200,Positive,... but... they're the cheesiest looking leath...,2014,06,12,0.0
10065,A2K5JT08E0WTWR,B00BKXN9LS,5.0,1373328000,Positive,Yep. it holds the strap on the neck of my guit...,2013,07,9,1.0
...,...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,5.0,1405814400,Positive,"Great, just as expected. Thank to all.Five Stars",2014,07,20,0.0
10257,A1RPTVW5VEOSI,B00JBIVXGC,5.0,1404259200,Positive,I've been thinking about trying the Nanoweb st...,2014,07,2,0.0
10258,AWCJ12KBO5VII,B00JBIVXGC,4.0,1405987200,Positive,I have tried coated strings in the past ( incl...,2014,07,22,0.0
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,4.0,1404172800,Positive,"Well, MADE by Elixir and DEVELOPED with Taylor...",2014,07,1,0.0


In [74]:
reviews['helpful_review'].value_counts()

0.00    7215
1.00    2040
0.50     266
0.67     136
0.75     111
        ... 
0.56       1
0.15       1
0.13       1
0.43       1
0.69       1
Name: helpful_review, Length: 65, dtype: int64

In [75]:
# Removing unnecessary column

reviews = reviews.drop(['unixReviewTime'], axis=1)

In [76]:
reviews.head()

,reviewerID,asin,overall,Sentiment,reviews,year,month,day,helpful_review
0,A2IBPI20UZIR0U,1384719342,5.0,Positive,"Not much to write about here, but it does exac...",2014,02,28,0.00
1,A14VAT5EAX3D9S,1384719342,5.0,Positive,The product does exactly as it should and is q...,2013,03,16,0.93
2,A195EZSQDW3E21,1384719342,5.0,Positive,The primary job of this device is to block the...,2013,08,28,1.00
3,A2C00NNG1ZQQG2,1384719342,5.0,Positive,Nice windscreen protects my MXL mic and preven...,2014,02,14,0.00
4,A94QU4C90B1AX,1384719342,5.0,Positive,This pop filter is great. It looks and perform...,2014,02,21,0.00


In [77]:
def clean_review(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

reviews['reviews'] = reviews['reviews'].apply(lambda x: clean_review(x))
reviews.head()

,reviewerID,asin,overall,Sentiment,reviews,year,month,day,helpful_review
0,A2IBPI20UZIR0U,1384719342,5.0,Positive,not much to write about here but it does exact...,2014,02,28,0.00
1,A14VAT5EAX3D9S,1384719342,5.0,Positive,the product does exactly as it should and is q...,2013,03,16,0.93
2,A195EZSQDW3E21,1384719342,5.0,Positive,the primary job of this device is to block the...,2013,08,28,1.00
3,A2C00NNG1ZQQG2,1384719342,5.0,Positive,nice windscreen protects my mxl mic and preven...,2014,02,14,0.00
4,A94QU4C90B1AX,1384719342,5.0,Positive,this pop filter is great it looks and performs...,2014,02,21,0.00
